In [ ]:
import json
import pandas as pd
import numpy as np
import plotly.express as px
import chart_studio.plotly as py
import plotly.graph_objs as go
from plotly import graph_objs as go

In [ ]:
churn_user_profile = pd.read_csv('Churn_UsersProfile.txt', delimiter = "|", encoding= 'unicode_escape')

churn_user_profile_copy = churn_user_profile.copy()

aggr_cols = [
        'Region_of_Origin',
        'District_of_Origin', 
        'County_of_Origin',
        'GenderDescription',
        'AgeClassDescription'
]

def aggregate_relative_change(churn_user_profile_copy, aggr_cols):
    tmp = churn_user_profile_copy.merge(
        churn_user_profile_copy, on=
            aggr_cols, how='left'
    )

    # Keep rows where period_x is different from period_y, and period_x is in 2019
    tmp = tmp[tmp.Period_x != tmp.Period_y]
    tmp = tmp[tmp.Period_x == 'Sep-19 to Feb-20']

    tmp['Relative_Change_AverageBusUsers'] = (tmp['Average_BusUsers_per_Day_y'] - tmp['Average_BusUsers_per_Day_x'])/(tmp['Average_BusUsers_per_Day_x'])
    
    return tmp

tmp = aggregate_relative_change(churn_user_profile_copy, aggr_cols)
daily_users_changes = tmp
tmp_mean = daily_users_changes.groupby('County_of_Origin')['Relative_Change_AverageBusUsers'].mean().reset_index()
tmp_median = daily_users_changes.groupby('County_of_Origin')['Relative_Change_AverageBusUsers'].median().reset_index()

tmp_mean.rename(columns = {'County_of_Origin':'NAME_2'}, inplace = True)
tmp_median.rename(columns = {'County_of_Origin':'NAME_2'}, inplace = True)

In [ ]:
# Opening JSON file
f = open('concelhos-shapefile.json',)
counties = json.load(f)

In [ ]:
#MAP MEAN

fig = px.choropleth_mapbox(data_frame=tmp_mean, geojson=counties, color='Relative_Change_AverageBusUsers',
                           locations='NAME_2', featureidkey="properties.NAME_2",
                           color_continuous_scale="Viridis",
                           range_color=(tmp_mean["Relative_Change_AverageBusUsers"].min(), tmp_mean["Relative_Change_AverageBusUsers"].max()),
                           center = {"lat": 38.736946, "lon": -9.142685}, 
                           opacity=0.5,
                           labels={"Relative_Change_AverageBusUsers":"Relative_Change_AverageBusUsers"},
                           mapbox_style="carto-positron", zoom=9)


fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
#MAP MEDIAN

fig = px.choropleth_mapbox(data_frame=tmp_median, geojson=counties, color='Relative_Change_AverageBusUsers',
                           locations='NAME_2', featureidkey="properties.NAME_2",
                           color_continuous_scale="Viridis",
                           range_color=(tmp_median["Relative_Change_AverageBusUsers"].min(), tmp_median["Relative_Change_AverageBusUsers"].max()),
                           center = {"lat": 38.736946, "lon": -9.142685}, 
                           opacity=0.5,
                           labels={"Relative_Change_AverageBusUsers":"Relative_Change_AverageBusUsers"},
                           mapbox_style="carto-positron", zoom=9)


fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()